In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

Imported all nessessary libraries

In [2]:
dir(transforms)

['AutoAugment',
 'AutoAugmentPolicy',
 'CenterCrop',
 'ColorJitter',
 'Compose',
 'ConvertImageDtype',
 'FiveCrop',
 'GaussianBlur',
 'Grayscale',
 'InterpolationMode',
 'Lambda',
 'LinearTransformation',
 'Normalize',
 'PILToTensor',
 'Pad',
 'RandomAdjustSharpness',
 'RandomAffine',
 'RandomApply',
 'RandomAutocontrast',
 'RandomChoice',
 'RandomCrop',
 'RandomEqualize',
 'RandomErasing',
 'RandomGrayscale',
 'RandomHorizontalFlip',
 'RandomInvert',
 'RandomOrder',
 'RandomPerspective',
 'RandomPosterize',
 'RandomResizedCrop',
 'RandomRotation',
 'RandomSizedCrop',
 'RandomSolarize',
 'RandomVerticalFlip',
 'Resize',
 'Scale',
 'TenCrop',
 'ToPILImage',
 'ToTensor',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'autoaugment',
 'functional',
 'functional_pil',
 'functional_tensor',
 'transforms']

In [3]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
])

In [4]:
dataset = torchvision.datasets.CIFAR10(root='./datasets/cifar10/train', download=True, transform = transform)

Files already downloaded and verified


In [5]:
dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./datasets/cifar10/train
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )

In [6]:
dataloader = torch.utils.data.DataLoader(dataset,
                                        batch_size=16,
                                        shuffle=True,
                                        num_workers=2)

In [7]:
data_mean = []
data_std = []

for i, data in enumerate(dataloader, 0):
    
    numpy_image = data[0].numpy()
    
    batch_mean = np.mean(numpy_image, axis=(0, 2, 3))
    batch_std = np.std(numpy_image, axis=(0, 2, 3))
    
    data_mean.append(batch_mean)
    data_std.append(batch_std)
    
data_mean = np.array(data_mean)
data_std = np.array(data_std)

data_mean = data_mean.mean(axis=0)
data_std = data_std.mean(axis=0)

In [8]:
print(data_mean)
print(data_std)

[0.49159086 0.48234665 0.44671988]
[0.2382715  0.23484378 0.25259557]


In [9]:
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(data_mean, data_std)
])

In [10]:
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(data_mean, data_std)
])

In [13]:
trainset = torchvision.datasets.CIFAR10(root='./datasets/cifar10/train', 
                                        train=True,
                                        download=True, 
                                        transform = train_transform)

Files already downloaded and verified


In [14]:
testset = torchvision.datasets.CIFAR10(root='./datasets/cifar10/train', 
                                        train=False,
                                        download=True, 
                                        transform = test_transform)

Files already downloaded and verified


In [15]:
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=16,
                                          shuffle=True,
                                          num_workers=2)

In [16]:
testloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=2)

In [17]:
num_channels = 3

hid1_size = 16
hid2_size = 32

out1_size = 400
out2_size = 10

kernel_size = 5

In [19]:
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(num_channels, hid1_size, kernel_size),
            nn.BatchNorm2d(hid1_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(hid1_size, hid2_size, kernel_size),
            nn.BatchNorm2d(hid2_size),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.linear1 = nn.Sequential(
            nn.Linear(hid2_size*kernel_size*kernel_size, out1_size),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(out1_size, out2_size)
        )
        
    def forward(self, x):
        
        out = self.conv1(x)
        out = self.conv2(out)
        
        out = out.reshape(out.size(0), -1)
        
        out = self.linear1(out)
        
        return F.softmax(out, dim=-1)

In [20]:
model = ConvNet()

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [22]:
print(device)

cpu


In [23]:
model.to(device)

ConvNet(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear1): Sequential(
    (0): Linear(in_features=800, out_features=400, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=400, out_features=10, bias=True)
  )
)

In [24]:
learning_rate = 0.0001

loss_metric = nn.NLLLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
total_step = len(trainloader)
num_epochs = 10
loss_values = list()

In [ ]:
for epoch in range(num_epochs):
    
    for i, (images, labels) in enumerate(trainloader):
        
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = loss_metric(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

print('Finished Training')

In [ ]:
model.eval()

with torch.no_grad():
    correct